<img src="img/diagrama_algoritmo.png", width=300, height=300>

SUPOSICIONES:
- Las cámaras de los drones tienen las mismas características y están posicionadas con el mismo ángulo de inclinación.
- Los drones vuelan a la misma altura y velocidad.


VERSION 0:
- A mayores, para una primera versión no se va a considerar la orientación del dron para tomar las imagenes porque se 
    supone que van a estar orientadas hacia el suelo.

In [ ]:
import numpy as np
from scipy.spatial import distance
import networkx as nx
import matplotlib.pyplot as plt
from geopy import distance

In [ ]:
altura_vuelo_uav = 20
ancho_sensor = 1
distancia_focal = 0.5
angulo_inclinacion = 90
fraccion_solape = 0.4

#Valores recibidos de NodeJS.
northWest = [42.17226800508796, -8.679393394103954] #(latitud, longitud)
northEast = [42.17226800508796, -8.678347434154489] #(latitud, longitud)
southWest = [42.17112037162631, -8.679393394103954] #(latitud, longitud)
southEast = [42.17112037162631, -8.678347434154489] #(latitud, longitud)

Calcula la distancia en metros entre dos coordenadas geográficas.

    Entradas:
        geoCoordA: (lat, long).
        geoCoordB: (lat, long).

    Salida:
        distance: (float), distancia en metros entre coordeadas geográficas.


In [ ]:
def getMeters(geoCoordA, geoCoordB):
    return distance.vincenty(geoCoordA,geoCoordB).meters

Calcula el número de metros que recoge la cámara del UAV.

    Entradas
        altura_vuelo_uav : (float), distancia en metros del dron al suelo del área que se quiere cubrir.
        ancho_sensor : (float), ancho del sensor de imagen en milimetros.
        distancia_focal : (float), distancia focal de la cámara en milimetros.
        angulo_inclinacion : (float), ángulo de inclinación de la cámara respecto al UAV en posición horizontal.

    Salida: 
        footprint : (float), número de metros capturados por la cámara del UAV.

In [ ]:
def calcularFootprint (altura_vuelo_uav, ancho_sensor, distancia_focal, angulo_inclinacion):
    # TODO: calcular para angulo de inclinación distinto de 90º
    return altura_vuelo_uav * (ancho_sensor / distancia_focal)

Calcula el número de filas necesarias para cubrir todo el área rectangular en sentido perpendicular al lado más corto respecto a la porción de área cubierta por la cámara y una fracción de solape.

    Entradas
        lado_area_corto : (float), longitud del lado más corto del área. Tiene que estar en las mismas unidades que 
        el footprint.                                
        footprint : (float), número de metros capturados por la cámara del UAV.
        fraccion_solape : (float), fracción de solape entre las imágenes tomadas por los UAVs.

    Salidas
        num_filas : (int), número de filas necesarias.
        lado_area_corto/num_filas : (float), distancia entre las filas.

In [3]:
def calcularNumeroFilasDistancia (lado_area_corto, footprint, fraccion_solape):
    num_filas = np.ceil(lado_area_corto / (footprint * (1 - fraccion_solape)))
    return int(num_filas), (lado_area_corto/num_filas)

Devuelve las divisiones en ambas cordenadas

In [ ]:
def calcularDivisionesArea (lado_area_corto, lado_area_largo, numero_filas, distancia_filas, footprint):
    numero_divisiones_lado_largo = int(np.ceil(lado_area_largo / footprint))

    divisiones_lado_corto = [i * distancia_filas - (distancia_filas / 2) for i in range (1, numero_filas + 1)]
    divisiones_lado_largo = [i * footprint - (footprint / 2) for i in range (1, numero_divisiones_lado_largo + 1)]

    return divisiones_lado_largo, divisiones_lado_corto

Devuelve una lista con las coordenadas de cada nodo del grafo.

In [ ]:
def calcularNodosGrafo (divisiones_x, divisiones_y):
    #return [(pos_x, pos_y) for pos_x in divisiones_x for pos_y in divisiones_y]
    nodos = []
    id = 1
    for pos_x in divisiones_x:
        fila_nodos = []
        for pos_y in divisiones_y:
            fila_nodos.append([id, (pos_x, pos_y)])
            id += 1

            # fila_nodos.append(np.array([pos_x, pos_y]))
            # fila_nodos.append((pos_x, pos_y))
        nodos.append(fila_nodos)
    return nodos

In [ ]:
def calcularGrafo (nodos):
    # Construimos el grafo con los nodos:
    grafo = nx.Graph()
    n_filas, n_columnas = len(nodos), len(nodos[0])
    for n_fila in range(n_filas):
        for n_columna in range (n_columnas):
            grafo.add_node(nodos[n_fila][n_columna][0], posicion = nodos[n_fila][n_columna][1], fila = n_fila, columna = n_columna)
            #grafo.add_node(nodos[n_fila][n_columna][0])

    # Establecemos los edges, lineas de conexion, entre los nodos:
    for n_fila in range(n_filas):
        for n_columna in range (n_columnas):
            for i, j in [[n_fila + 1, n_columna], [n_fila - 1, n_columna], [n_fila, n_columna + 1], [n_fila, n_columna - 1]]:
                if i < 0 or j < 0 or i >= n_filas or j >= n_columnas: # No existe el nodo
                    continue
                grafo.add_edge(nodos[n_fila][n_columna][0], nodos[i][j][0]) # nodos[n_fila][n_columna][0] ----(vecino)---- nodos[i][j][0]
    return grafo

Convierte coordenadas cartesianas a coordenadas geométricas

        Entradas
            x_meters: (float), indica cuántos metros en el eje x nos desplazamos con respecto al origen de 
            coordenadas.
            y_meters: (float), indica cuántos metros en el eje y nos desplazamos con respecto al origen de 
            coordenadas.
            origenCoordGeo: (lat,long), indica cual es el punto de origen de coordenadas, el cuál sirve para 
            relaccionar la posición real de la zona a cubrir con la distancia en metros teórica hasta el 
            siguiente destino.
        
        Salidas
            latitud: (float), latitud real del punto desplazado los metros desados en el eje x
            longitud: (float), longitud real del punto desplazado los metros desados en el eje y

In [ ]:
def cartToGeoCoord(x_meters, y_meters, origenCoordGeo):
    geoCoord = distance.vincenty(meters=x_meters).destination(origenCoordGeo, 90)
    geoCoord = distance.vincenty(meters=y_meters).destination(geoCoord, 0)
    
    return geoCoord.latitude, geoCoord.longitude
    

In [ ]:

if __name__ == '__main__':
    # def calcularGrafo (x, y, altura_vuelo_uav, caracteristicas_sensor):
    x = getMeters(southWest,southEast)
    y = getMeters(southWest,northWest)
    lado_area_corto = x * (x < y) + y * (y < x) + x * (x == y)
    lado_area_largo = x * (x > y) + y * (y > x) + x * (x == y)

    footprint = calcularFootprint (altura_vuelo_uav, ancho_sensor, distancia_focal, angulo_inclinacion)
    numero_filas, distancia_filas = calcularNumeroFilasDistancia (lado_area_corto, footprint, fraccion_solape)

    if lado_area_largo == x:
        divisiones_x, divisiones_y = calcularDivisionesArea (lado_area_corto, lado_area_largo, numero_filas, distancia_filas, footprint)
    else:
        divisiones_y, divisiones_x = calcularDivisionesArea (lado_area_corto, lado_area_largo, numero_filas, distancia_filas, footprint)

    nodos = calcularNodosGrafo (divisiones_x, divisiones_y)
    grafo = calcularGrafo(nodos)

    
    #Pasamos la primera coordenada cartesiana a coordenadas geometricas
    coordCart = nodos[0][0][1];
    coordGeo = cartToGeoCoord(coordCart[0],0,southWest)
    
    print coordCart
    print coordGeo
    print getMeters(geoCoordA=southWest, geoCoordB=coordGeo)
    
    
    
    